In [8]:
using Turing
using LimberJack
using LinearAlgebra
using CSV
using NPZ
using PythonCall
using PythonCall
np = pyimport("numpy")

Python module: <module 'numpy' from '/home/jaime/.julia/environments/v1.7/.CondaPkg/env/lib/python3.10/site-packages/numpy/__init__.py'>

# Make N(z)'s 

In [4]:
nz_path = "../data/DESY1/fullmarg_nzs/" 
zs_k0, nz_k0, cov_k0 = get_nzs(nz_path, "20")
zs_k1, nz_k1, cov_k1 = get_nzs(nz_path, "21")
zs_k2, nz_k2, cov_k2 = get_nzs(nz_path, "22")
zs_k3, nz_k3, cov_k3 = get_nzs(nz_path, "23")

(Float32[0.02, 0.06, 0.099999994, 0.14, 0.18, 0.22, 0.26, 0.3, 0.34, 0.38  …  3.62, 3.66, 3.7, 3.74, 3.78, 3.82, 3.8600001, 3.9, 3.94, 3.98], [0.0, 0.005984172563510772, 0.03765643218161212, 0.028133875010282052, 0.026655211697212404, 0.06408162232970639, 0.02553711344743602, 0.03706050250131086, 0.0216295539779018, 0.04669891197796871  …  0.0, 0.0, 0.0, 0.0006562865728945231, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.6708037247042034e-8 5.802577730455622e-5 … 0.0 0.0; 0.0 5.804248534180326e-5 … 0.0 0.0; … ; 0.0 5.802577730455622e-5 … 1.6708037247042034e-8 0.0; 0.0 5.802577730455622e-5 … 0.0 1.6708037247042034e-8])

In [5]:
err_k0 = sqrt.(diag(cov_k0))
err_k1 = sqrt.(diag(cov_k1))
err_k2 = sqrt.(diag(cov_k2))
err_k3 = sqrt.(diag(cov_k3));

In [368]:
using Interpolations

In [473]:
l_X = LinRange(0, 1.1, 20)
X = exp.(l_X) .- 1;

In [474]:
nz_k0_itp = LinearInterpolation(zs_k0, nz_k0, extrapolation_bc=Line())
err_k0_itp = LinearInterpolation(zs_k0, err_k0, extrapolation_bc=Line())

nz_k1_itp = LinearInterpolation(zs_k1, nz_k1, extrapolation_bc=Line())
err_k1_itp = LinearInterpolation(zs_k1, err_k1, extrapolation_bc=Line());

nz_k2_itp = LinearInterpolation(zs_k2, nz_k2, extrapolation_bc=Line())
err_k2_itp = LinearInterpolation(zs_k2, err_k2, extrapolation_bc=Line());

nz_k3_itp = LinearInterpolation(zs_k3, nz_k3, extrapolation_bc=Line())
err_k3_itp = LinearInterpolation(zs_k3, err_k3, extrapolation_bc=Line());

In [476]:
nz_k0_lite = nz_k0_itp(X) 
err_k0_lite = err_k0_itp(X).^2
cov_k0_lite = err_k0_lite.^2 

nz_k1_lite = nz_k1_itp(X) 
err_k1_lite = err_k1_itp(X)
cov_k1_lite = err_k1_lite.^2 

nz_k2_lite = nz_k2_itp(X)
err_k2_lite = err_k2_itp(X).^2 
cov_k2_lite = err_k2_lite.^2 

nz_k3_lite = nz_k3_itp(X)
err_k3_lite = err_k3_itp(X).^2 
cov_k3_lite = err_k3_lite.^2;

In [479]:
npzwrite("../data/DESY1/lite20_nzs/nz_DESwl__0_e.npz", Dict("z" => X, "dndz"=> nz_k0_lite, "cov"=> cov_k0_lite))
npzwrite("../data/DESY1/lite20_nzs/nz_DESwl__1_e.npz", Dict("z" => X, "dndz"=> nz_k1_lite, "cov"=> cov_k1_lite))
npzwrite("../data/DESY1/lite20_nzs/nz_DESwl__2_e.npz", Dict("z" => X, "dndz"=> nz_k2_lite, "cov"=> cov_k2_lite))
npzwrite("../data/DESY1/lite20_nzs/nz_DESwl__3_e.npz", Dict("z" => X, "dndz"=> nz_k3_lite, "cov"=> cov_k3_lite))

# Lkl

In [13]:
fol = "DESY1"
data_set = "wlwl_lite10"
meta = np.load(string("../data/", fol, "/", data_set, "_meta.npz"))
files = npzread(string("../data/", fol, "/", data_set, "_files.npz"))

Dict{String, Array{Float64}} with 14 entries:
  "ls_DESwl__2_e_DESwl__3_e" => [44.5, 74.5, 104.5, 134.5, 164.5, 194.5, 224.5,…
  "ls_DESwl__3_e_DESwl__3_e" => [44.5, 74.5, 104.5, 134.5, 164.5, 194.5, 224.5,…
  "ls_DESwl__0_e_DESwl__2_e" => [44.5, 74.5, 104.5, 134.5, 164.5, 194.5, 224.5,…
  "ls_DESwl__0_e_DESwl__3_e" => [44.5, 74.5, 104.5, 134.5, 164.5, 194.5, 224.5,…
  "ls_DESwl__0_e_DESwl__0_e" => [44.5, 74.5, 104.5, 134.5, 164.5, 194.5, 224.5,…
  "ls_DESwl__1_e_DESwl__2_e" => [44.5, 74.5, 104.5, 134.5, 164.5, 194.5, 224.5,…
  "nz_DESwl__0_e"            => [0.0 0.130005 … 1.65854 2.00417; -0.0156303 0.2…
  "ls_DESwl__1_e_DESwl__1_e" => [44.5, 74.5, 104.5, 134.5, 164.5, 194.5, 224.5,…
  "ls_DESwl__0_e_DESwl__1_e" => [44.5, 74.5, 104.5, 134.5, 164.5, 194.5, 224.5,…
  "ls_DESwl__2_e_DESwl__2_e" => [44.5, 74.5, 104.5, 134.5, 164.5, 194.5, 224.5,…
  "nz_DESwl__1_e"            => [0.0 0.130005 … 1.65854 2.00417; -0.0110362 0.1…
  "ls_DESwl__1_e_DESwl__3_e" => [44.5, 74.5, 104.5, 134.5, 164.

In [14]:
tracers_names = pyconvert(Vector{String}, meta["tracers"])
pairs = pyconvert(Vector{Vector{String}}, meta["pairs"]);
pairs_ids = pyconvert(Vector{Vector{Int}}, meta["pairs_ids"])
idx = pyconvert(Vector{Int}, meta["idx"])
data_vector = pyconvert(Vector{Float64}, meta["cls"])
cov_tot = pyconvert(Matrix{Float64}, meta["cov"]);

nz_path = "../data/DESY1/lite10_nzs/"
zs_k0, nz_k0, cov_k0 = get_nzs(nz_path, "DESwl__0_e")
zs_k1, nz_k1, cov_k1 = get_nzs(nz_path, "DESwl__1_e")
zs_k2, nz_k2, cov_k2 = get_nzs(nz_path, "DESwl__2_e")
zs_k3, nz_k3, cov_k3 = get_nzs(nz_path, "DESwl__3_e")

([0.0, 0.13000518614461032, 0.2769117207137157, 0.44291686665533714, 0.6305035424960621, 0.8424774590477093, 1.0820090840784555, 1.3526810626088452, 1.6585418020922078, 2.0041660239464334], [-0.0029920862817553865, 0.03051327942103319, 0.03040912201514251, 0.013837779618869402, 0.024305171252077012, 0.44232303886198976, 0.30477089069887153, 0.080983405477799, 0.023554788326521005, 0.002940391214619623], [1.70852513625414e-10, 2.831002605788785e-7, 1.2031648937001716e-7, 1.0115650342716598e-8, 5.035616234807032e-8, 0.0011019940755240378, 0.0001707677072101776, 9.284305121737408e-7, 2.51412459131466e-8, 2.3311197002382846e-10])

In [20]:
@model function model(data_vector;
                      tracers_names=tracers_names,
                      pairs=pairs,
                      pairs_id=pairs_ids,
                      idx=idx,
                      cov_tot=cov_tot, 
                      files=files)

    #KiDS priors
    Ωm ~ Uniform(0.2, 0.6)
    Ωb ~ Uniform(0.028, 0.065)
    h ~ Uniform(0.64, 0.82)
    s8 ~ Uniform(0.6, 0.9)
    ns ~ Uniform(0.84, 1.1)
    
    A_IA ~ Uniform(-5, 5) 
    alpha_IA ~ Uniform(-5, 5)

    DESwl__0_e_nz ~ MvNormal(nz_k0, cov_k0)
    DESwl__1_e_nz ~ MvNormal(nz_k1, cov_k1)
    DESwl__2_e_nz ~ MvNormal(nz_k2, cov_k2)
    DESwl__3_e_nz ~ MvNormal(nz_k3, cov_k3)
    DESwl__0_e_m ~ Normal(0.012, 0.023)
    DESwl__1_e_m ~ Normal(0.012, 0.023)
    DESwl__2_e_m ~ Normal(0.012, 0.023)
    DESwl__3_e_m ~ Normal(0.012, 0.023)


    nuisances = Dict("A_IA" => A_IA,
                     "alpha_IA" => alpha_IA,

                     "DESwl__0_e_nz" => DESwl__0_e_nz,
                     "DESwl__1_e_nz" => DESwl__1_e_nz,
                     "DESwl__2_e_nz" => DESwl__2_e_nz,
                     "DESwl__3_e_nz" => DESwl__3_e_nz,
                     "DESwl__0_e_m" => DESwl__0_e_m,
                     "DESwl__1_e_m" => DESwl__1_e_m,
                     "DESwl__2_e_m" => DESwl__2_e_m,
                     "DESwl__3_e_m" => DESwl__3_e_m)

    
    cosmology = LimberJack.Cosmology(Ωm, Ωb, h, ns, s8,
                                     tk_mode="EisHu",
                                     Pk_mode="Halofit")
    
    theory = Theory(cosmology, tracers_names, pairs,
                    pairs_ids, idx, files;
                    Nuisances=nuisances)
    data_vector ~ MvNormal(theory, cov_tot)
end;

In [21]:
stat_model = model(data_vector);

In [ ]:
chain = sample(stat_model, NUTS(3, 0.6), 100,
                progress=true; save_state=true);

In [32]:
chain1 = sample(model(data_vector), MH(), iterations,
            nadapt=nadapts, progress=true; save_state=true);

LoadError: MethodError: no method matching PDMats.PDMat(::Hermitian{Float64, Matrix{Float64}})
[0mClosest candidates are:
[0m  PDMats.PDMat(::AbstractMatrix, [91m::Cholesky{T, S}[39m) where {T, S} at ~/.julia/packages/PDMats/G0Prn/src/pdmat.jl:12
[0m  PDMats.PDMat([91m::Cholesky[39m) at ~/.julia/packages/PDMats/G0Prn/src/pdmat.jl:21
[0m  PDMats.PDMat([91m::Symmetric[39m) at ~/.julia/packages/PDMats/G0Prn/src/pdmat.jl:20
[0m  ...